In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time
import numpy as np

In [2]:
# baseline model with dropout and data augmentation on the cifar10 dataset
import sys
from matplotlib import pyplot
from keras.datasets import cifar10
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
from keras.layers import BatchNormalization

In [3]:
# load train and test dataset
def load_dataset():
    # load dataset
    data_x = np.load('data_x_original.npy')
    data_y = np.load('data_y_original.npy')
    #percentage of the dataset which will be on TRAIN - TEST - VALIDATION
    TOTAL_IMAGES = len(data_y)
    print("TOTAL_IMAGES = ", TOTAL_IMAGES)
    
    TRAIN_PERCENTAGE = 0.8
    TEST_PERCENTAGE = 0.1
    VALIDATION_PERCENTAGE = 0.1

    train_size = int(TRAIN_PERCENTAGE * TOTAL_IMAGES)
    test_size = int(TEST_PERCENTAGE * TOTAL_IMAGES)
    validation_size = int(VALIDATION_PERCENTAGE * TOTAL_IMAGES)

    train_x = data_x[:train_size] #taking train images from original data
    train_y = data_y[:train_size]
    data_x = data_x[train_size:] #removing train images from original data
    data_y = data_y[train_size:]

    test_x = data_x[:test_size] #taking test images from original data
    test_y = data_y[:test_size]
    data_x = data_x[test_size:] #removing test images from original data
    data_y = data_y[test_size:]

    validation_x = data_x #all the remainder images we can put on validation
    validation_y = data_y

    del data_x #free some unused memory
    del data_y

    print("train_size = ", len(train_y))
    print("test_size = ", len(test_y))
    print("validation_size = ", len(validation_y))
 # one hot encode target values
    train_y = to_categorical(train_y)
    test_y = to_categorical(test_y)
    validation_y = to_categorical(validation_y)
    return train_x, train_y, test_x, test_y, validation_x, validation_y

In [4]:
# scale pixels
def prep_pixels(train, test, validation):
 # convert from integers to floats
 train_norm = train.astype('float32')
 test_norm = test.astype('float32')
 validation_norm = validation.astype('float32')
 # normalize to range 0-1
 train_norm = train_norm / 255.0
 test_norm = test_norm / 255.0
 validation_norm = validation_norm / 255.0
 # return normalized images
 return train_norm, test_norm, validation_norm

In [5]:
# define cnn model
def define_model():
    new_model = Sequential()
    
    model = tf.keras.applications.ResNet50V2(
        include_top=False,
        weights="imagenet",
        input_tensor=None,
        input_shape=(128,128,3),
        pooling=None,
        classes=4,
       # classifier_activation="softmax",
    #    **kwargs
        )
    
    new_model.add(model)
    #3rd conv layer
    new_model.add(tf.keras.layers.Conv2D(32, 3, padding="same"))
    new_model.add(tf.keras.layers.BatchNormalization())
    new_model.add(tf.keras.layers.Activation("relu"))
    new_model.add(tf.keras.layers.MaxPooling2D(2, padding='same'))
    new_model.add(tf.keras.layers.Dropout(0.4))
    #4th conv layer
    new_model.add(tf.keras.layers.Conv2D(64, 3, padding="same"))
    new_model.add(tf.keras.layers.BatchNormalization())
    new_model.add(tf.keras.layers.Activation("relu"))
    new_model.add(tf.keras.layers.MaxPooling2D(2, padding='same'))
    new_model.add(tf.keras.layers.Dropout(0.3))
    #Flatten Layer
    new_model.add(tf.keras.layers.Flatten())

    new_model.add(tf.keras.layers.Dense(128))
    new_model.add(tf.keras.layers.BatchNormalization())
    new_model.add(tf.keras.layers.Activation("relu"))
    new_model.add(tf.keras.layers.Dropout(0.2))
    #output Dense Layer
    new_model.add(tf.keras.layers.Dense(4))
    new_model.add(tf.keras.layers.Activation('softmax'))
    opt = SGD(lr=0.0001, momentum=0.9)
    new_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return new_model

In [6]:
# plot diagnostic learning curves
def summarize_diagnostics(history):
 # plot loss
    model_trained = history
    accuracy = model_trained.history['accuracy']
    val_accuracy = model_trained.history['val_accuracy']
    loss = model_trained.history['loss']
    val_loss = model_trained.history['val_loss']
    epochs = range(len(accuracy))
    plt.plot(epochs, accuracy, 'b', label='Acurácia de treino')
    plt.plot(epochs, val_accuracy, 'g', label='Acurácia de validação')
    plt.title('Acurácia de treino e validação')
    plt.legend()
    plt.figure()
    plt.plot(epochs, loss, 'b', label='Perda no treino')
    plt.plot(epochs, val_loss, 'g', label='Perda na validação')
    plt.title('Perda na validação e treino')
    plt.legend()
    plt.show()

In [7]:
epochs = 400
verb = 2

In [ ]:
trainX, trainY, testX, testY, validationX, validationY = load_dataset()
print(trainY)
 # prepare pixel data
trainX, testX, validationX = prep_pixels(trainX, testX, validationX)
 # define model
model = define_model()
datagen = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
 # prepare iterator
it_train = datagen.flow(trainX, trainY, batch_size=64)
 # fit model
steps = int(trainX.shape[0] / 64)
history = model.fit(it_train, steps_per_epoch=steps, epochs=epochs, validation_data=(validationX, validationY), verbose=verb)


TOTAL_IMAGES =  592
train_size =  473
test_size =  59
validation_size =  60
[[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 ...
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]


C:\Users\tmcle\.conda\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/400
7/7 - 18s - loss: 1.7612 - accuracy: 0.2494 - val_loss: 1.9238 - val_accuracy: 0.1833 - 18s/epoch - 3s/step
Epoch 2/400
7/7 - 3s - loss: 1.7748 - accuracy: 0.2421 - val_loss: 1.8216 - val_accuracy: 0.2000 - 3s/epoch - 470ms/step
Epoch 3/400
7/7 - 3s - loss: 1.7747 - accuracy: 0.2763 - val_loss: 1.7651 - val_accuracy: 0.2000 - 3s/epoch - 478ms/step
Epoch 4/400
7/7 - 4s - loss: 1.8044 - accuracy: 0.2522 - val_loss: 1.7114 - val_accuracy: 0.1667 - 4s/epoch - 504ms/step
Epoch 5/400
7/7 - 3s - loss: 1.6895 - accuracy: 0.2910 - val_loss: 1.6709 - val_accuracy: 0.1833 - 3s/epoch - 479ms/step
Epoch 6/400
7/7 - 4s - loss: 1.7302 - accuracy: 0.2634 - val_loss: 1.6267 - val_accuracy: 0.1833 - 4s/epoch - 508ms/step
Epoch 7/400
7/7 - 3s - loss: 1.7287 - accuracy: 0.2372 - val_loss: 1.5696 - val_accuracy: 0.2167 - 3s/epoch - 474ms/step
Epoch 8/400
7/7 - 3s - loss: 1.6648 - accuracy: 0.2714 - val_loss: 1.5566 - val_accuracy: 0.2167 - 3s/epoch - 473ms/step
Epoch 9/400
7/7 - 3s - loss: 1.69

Epoch 69/400
7/7 - 3s - loss: 0.9022 - accuracy: 0.6259 - val_loss: 0.9207 - val_accuracy: 0.6667 - 3s/epoch - 476ms/step
Epoch 70/400
7/7 - 4s - loss: 0.9371 - accuracy: 0.6272 - val_loss: 0.9174 - val_accuracy: 0.6833 - 4s/epoch - 506ms/step
Epoch 71/400
7/7 - 3s - loss: 0.8339 - accuracy: 0.6626 - val_loss: 0.9098 - val_accuracy: 0.6833 - 3s/epoch - 477ms/step
Epoch 72/400
7/7 - 3s - loss: 0.8999 - accuracy: 0.6308 - val_loss: 0.9080 - val_accuracy: 0.6667 - 3s/epoch - 476ms/step
Epoch 73/400
7/7 - 3s - loss: 0.8887 - accuracy: 0.6308 - val_loss: 0.9081 - val_accuracy: 0.6500 - 3s/epoch - 478ms/step
Epoch 74/400
7/7 - 3s - loss: 0.8756 - accuracy: 0.6235 - val_loss: 0.8898 - val_accuracy: 0.6667 - 3s/epoch - 477ms/step
Epoch 75/400
7/7 - 3s - loss: 0.8509 - accuracy: 0.6846 - val_loss: 0.8748 - val_accuracy: 0.6833 - 3s/epoch - 474ms/step
Epoch 76/400
7/7 - 3s - loss: 0.8412 - accuracy: 0.6601 - val_loss: 0.8639 - val_accuracy: 0.6833 - 3s/epoch - 474ms/step
Epoch 77/400
7/7 - 4s - 

Epoch 136/400
7/7 - 3s - loss: 0.5807 - accuracy: 0.8117 - val_loss: 0.7004 - val_accuracy: 0.8000 - 3s/epoch - 479ms/step
Epoch 137/400
7/7 - 3s - loss: 0.5798 - accuracy: 0.7800 - val_loss: 0.6767 - val_accuracy: 0.8167 - 3s/epoch - 486ms/step
Epoch 138/400
7/7 - 3s - loss: 0.5739 - accuracy: 0.8117 - val_loss: 0.6698 - val_accuracy: 0.8167 - 3s/epoch - 477ms/step
Epoch 139/400
7/7 - 3s - loss: 0.5561 - accuracy: 0.7971 - val_loss: 0.6923 - val_accuracy: 0.7667 - 3s/epoch - 479ms/step
Epoch 140/400
7/7 - 3s - loss: 0.5483 - accuracy: 0.8117 - val_loss: 0.6747 - val_accuracy: 0.8000 - 3s/epoch - 482ms/step
Epoch 141/400
7/7 - 3s - loss: 0.4927 - accuracy: 0.8191 - val_loss: 0.6734 - val_accuracy: 0.7833 - 3s/epoch - 478ms/step
Epoch 142/400
7/7 - 3s - loss: 0.5052 - accuracy: 0.8460 - val_loss: 0.6907 - val_accuracy: 0.8000 - 3s/epoch - 480ms/step
Epoch 143/400
7/7 - 3s - loss: 0.5535 - accuracy: 0.7922 - val_loss: 0.6997 - val_accuracy: 0.8000 - 3s/epoch - 478ms/step
Epoch 144/400
7/

Epoch 203/400
7/7 - 3s - loss: 0.2983 - accuracy: 0.9242 - val_loss: 0.5968 - val_accuracy: 0.8167 - 3s/epoch - 484ms/step
Epoch 204/400
7/7 - 3s - loss: 0.3515 - accuracy: 0.8973 - val_loss: 0.5993 - val_accuracy: 0.8333 - 3s/epoch - 482ms/step
Epoch 205/400
7/7 - 3s - loss: 0.2730 - accuracy: 0.9389 - val_loss: 0.6083 - val_accuracy: 0.8000 - 3s/epoch - 481ms/step
Epoch 206/400
7/7 - 3s - loss: 0.3055 - accuracy: 0.9071 - val_loss: 0.6219 - val_accuracy: 0.8000 - 3s/epoch - 468ms/step
Epoch 207/400
7/7 - 3s - loss: 0.3073 - accuracy: 0.9315 - val_loss: 0.6169 - val_accuracy: 0.8167 - 3s/epoch - 480ms/step
Epoch 208/400
7/7 - 3s - loss: 0.2750 - accuracy: 0.9242 - val_loss: 0.6083 - val_accuracy: 0.8167 - 3s/epoch - 481ms/step
Epoch 209/400
7/7 - 3s - loss: 0.3013 - accuracy: 0.9193 - val_loss: 0.6069 - val_accuracy: 0.8167 - 3s/epoch - 484ms/step
Epoch 210/400
7/7 - 3s - loss: 0.3077 - accuracy: 0.9193 - val_loss: 0.6063 - val_accuracy: 0.8167 - 3s/epoch - 481ms/step
Epoch 211/400
7/

Epoch 270/400
7/7 - 3s - loss: 0.2159 - accuracy: 0.9438 - val_loss: 0.6848 - val_accuracy: 0.7667 - 3s/epoch - 486ms/step
Epoch 271/400
7/7 - 4s - loss: 0.1831 - accuracy: 0.9665 - val_loss: 0.6891 - val_accuracy: 0.7667 - 4s/epoch - 516ms/step
Epoch 272/400
7/7 - 3s - loss: 0.2183 - accuracy: 0.9438 - val_loss: 0.6872 - val_accuracy: 0.7667 - 3s/epoch - 483ms/step
Epoch 273/400
7/7 - 3s - loss: 0.1773 - accuracy: 0.9609 - val_loss: 0.6648 - val_accuracy: 0.7667 - 3s/epoch - 484ms/step
Epoch 274/400
7/7 - 3s - loss: 0.2003 - accuracy: 0.9707 - val_loss: 0.6777 - val_accuracy: 0.7667 - 3s/epoch - 483ms/step
Epoch 275/400
7/7 - 3s - loss: 0.1883 - accuracy: 0.9560 - val_loss: 0.6992 - val_accuracy: 0.7667 - 3s/epoch - 485ms/step
Epoch 276/400
7/7 - 3s - loss: 0.1903 - accuracy: 0.9511 - val_loss: 0.7128 - val_accuracy: 0.7667 - 3s/epoch - 471ms/step
Epoch 277/400
7/7 - 3s - loss: 0.1950 - accuracy: 0.9560 - val_loss: 0.6845 - val_accuracy: 0.7667 - 3s/epoch - 484ms/step
Epoch 278/400
7/

Epoch 337/400
7/7 - 3s - loss: 0.1304 - accuracy: 0.9853 - val_loss: 0.6585 - val_accuracy: 0.7833 - 3s/epoch - 469ms/step
Epoch 338/400
7/7 - 3s - loss: 0.1354 - accuracy: 0.9584 - val_loss: 0.6519 - val_accuracy: 0.8000 - 3s/epoch - 483ms/step
Epoch 339/400
7/7 - 3s - loss: 0.1128 - accuracy: 0.9902 - val_loss: 0.6659 - val_accuracy: 0.8000 - 3s/epoch - 483ms/step
Epoch 340/400
7/7 - 3s - loss: 0.1362 - accuracy: 0.9682 - val_loss: 0.6779 - val_accuracy: 0.8000 - 3s/epoch - 484ms/step
Epoch 341/400
7/7 - 3s - loss: 0.1225 - accuracy: 0.9902 - val_loss: 0.6933 - val_accuracy: 0.8000 - 3s/epoch - 471ms/step
Epoch 342/400
7/7 - 4s - loss: 0.1297 - accuracy: 0.9799 - val_loss: 0.6870 - val_accuracy: 0.8000 - 4s/epoch - 518ms/step
Epoch 343/400
7/7 - 3s - loss: 0.1491 - accuracy: 0.9633 - val_loss: 0.6897 - val_accuracy: 0.8000 - 3s/epoch - 483ms/step
Epoch 344/400
7/7 - 3s - loss: 0.1732 - accuracy: 0.9560 - val_loss: 0.6788 - val_accuracy: 0.7833 - 3s/epoch - 486ms/step
Epoch 345/400
7/

In [ ]:
 # evaluate model
_, acc = model.evaluate(testX, testY, verbose=verb)
print('Acurácia: %.3f' % (acc * 100.0))

In [ ]:
 # learning curves
summarize_diagnostics(history)

In [ ]:
model_name = f'displasys-resnet50v2-baseline-holdout-{epochs}epochs_transferlearning.npy'
model.save(model_name)

In [20]:
# entry point, run the test harness
#run_test_harness(epochs=100, verb=1)